# Read DineSafe Open Data for Grocery Store Locations

Data on DineSafe inspections are provided on the City of Toronto's OpenData website. 

This script provides an example of how to read this data in using pandas, and then save as a shapefile for future processing.

In this example we'll use all supermarkets (even 7-Elevens) to be consistent with DineSafe's categorization. 

In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

In [ ]:
root_dir = Path(r"C:\Access To Opportunities")
data_root_dir = root_dir / "Data"

# DineSafe establishments, as downloaded from City of Toronto Open Data
dinesafe_fp = data_root_dir / "Dinesafe" / "2024-02-06" / "Dinesafe.csv"

# Create a working directory to hold files
working_dir = root_dir / "working"
working_dir.mkdir(exist_ok=True)

wgs84_proj_str = "EPSG:4326"
cot_proj_str = "EPSG:2952"

In [ ]:
# Read DineSafe data into pandas DataFrame
df = pd.read_csv(dinesafe_fp, usecols=['Establishment ID', 'Establishment Name', 'Establishment Type', 'Latitude', 'Longitude'])

In [ ]:
# Filter out all records that are not supermarkets
df = df.loc[df['Establishment Type'] == 'Supermarket']

In [ ]:
# Remove duplicates to keep only one record per establishment
df = df.drop_duplicates(subset=['Establishment ID'])

In [ ]:
# Rename columns to keep within 10-character limit of shapefiles
df = df.rename({'Establishment ID': 'id', 'Establishment Name': 'name', 'Establishment Type': 'type'}, axis=1)

In [ ]:
# Convert to a pandas GeoDataFrame
gdf = gpd.GeoDataFrame(
    df, 
    geometry=gpd.points_from_xy(df.Longitude, df.Latitude), 
    crs="EPSG:4326"
)

In [ ]:
# We don't need the original latitude and longitude columns anymore
gdf = gdf.drop(['Latitude', 'Longitude'], axis=1)

In [ ]:
# Project to the City of Toronto's preferred projection system
gdf = gdf.to_crs(cot_proj_str)

In [ ]:
# Export to shapefile
gdf.to_file(working_dir / "dinesafe_supermarkets.shp", index=False)